# Sqlite Clase 18-06-2025

In [34]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [35]:
DF= pd.read_json('https://www.datos.gov.co/resource/d7zw-hpf4.json?$limit=100000')

DF

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,tipo_delito,cantidad
0,1996-01-01T00:00:00.000,5,ANTIOQUIA,5042,SANTAFE DE ANTIOQUIA,SECUESTRO SIMPLE,1
1,1996-01-01T00:00:00.000,25,CUNDINAMARCA,25290,FUSAGASUGA,SECUESTRO EXTORSIVO,1
2,1996-01-02T00:00:00.000,5,ANTIOQUIA,5038,ANGOSTURA,SECUESTRO EXTORSIVO,1
3,1996-01-02T00:00:00.000,68,SANTANDER,68001,BUCARAMANGA,SECUESTRO EXTORSIVO,1
4,1996-01-02T00:00:00.000,5,ANTIOQUIA,5088,BELLO,SECUESTRO EXTORSIVO,1
...,...,...,...,...,...,...,...
28015,2025-04-16T00:00:00.000,5,ANTIOQUIA,5440,MARINILLA,SECUESTRO EXTORSIVO,1
28016,2025-04-16T00:00:00.000,5,ANTIOQUIA,5440,MARINILLA,SECUESTRO EXTORSIVO,1
28017,2025-04-16T00:00:00.000,19,CAUCA,19001,POPAYAN,SECUESTRO EXTORSIVO,1
28018,2025-04-16T00:00:00.000,19,CAUCA,19001,POPAYAN,SECUESTRO EXTORSIVO,1


In [36]:
# Crear conexión a la base de datos
conn = sqlite3.connect('../Datos/secuestros.db')
cursor = conn.cursor()

# Función auxiliar para ejecutar consultas
def ejecutar_sql(query, conexion=conn):
    """
    Ejecuta una consulta SQL y devuelve el resultado en un DataFrame.
    """
    df = pd.read_sql_query(query, conexion)
    display(df)  # Muestra bonita si estás en Colab o Jupyter


In [37]:
# Guardar la tabla
DF.to_sql('secuestros', conn, if_exists='replace', index=False)


28020

In [38]:
# Ver los nombres de las columnas
columnas = pd.read_sql_query("PRAGMA table_info(secuestros)", conn)
print(columnas)


   cid          name     type  notnull dflt_value  pk
0    0   fecha_hecho     TEXT        0       None   0
1    1     cod_depto  INTEGER        0       None   0
2    2  departamento     TEXT        0       None   0
3    3      cod_muni  INTEGER        0       None   0
4    4     municipio     TEXT        0       None   0
5    5   tipo_delito     TEXT        0       None   0
6    6      cantidad  INTEGER        0       None   0


In [39]:
# Ver las primeras filas
ejecutar_sql("SELECT * FROM secuestros LIMIT 5")


,fecha_hecho,cod_depto,departamento,cod_muni,municipio,tipo_delito,cantidad
0,1996-01-01T00:00:00.000,5,ANTIOQUIA,5042,SANTAFE DE ANTIOQUIA,SECUESTRO SIMPLE,1
1,1996-01-01T00:00:00.000,25,CUNDINAMARCA,25290,FUSAGASUGA,SECUESTRO EXTORSIVO,1
2,1996-01-02T00:00:00.000,5,ANTIOQUIA,5038,ANGOSTURA,SECUESTRO EXTORSIVO,1
3,1996-01-02T00:00:00.000,68,SANTANDER,68001,BUCARAMANGA,SECUESTRO EXTORSIVO,1
4,1996-01-02T00:00:00.000,5,ANTIOQUIA,5088,BELLO,SECUESTRO EXTORSIVO,1


In [40]:
# Contar registros
ejecutar_sql("SELECT COUNT(*) as total_registros FROM secuestros")


,total_registros
0,28020


In [41]:

# Ver columnas
ejecutar_sql("PRAGMA table_info(secuestros)")

,cid,name,type,notnull,dflt_value,pk
0,0,fecha_hecho,TEXT,0,None,0
1,1,cod_depto,INTEGER,0,None,0
2,2,departamento,TEXT,0,None,0
3,3,cod_muni,INTEGER,0,None,0
4,4,municipio,TEXT,0,None,0
5,5,tipo_delito,TEXT,0,None,0
6,6,cantidad,INTEGER,0,None,0


In [42]:
# Extraer el año con pandas
DF['aÑo'] = pd.to_datetime(DF['fecha_hecho'], errors='coerce').dt.year

# Guardar de nuevo en SQLite incluyendo el año
DF.to_sql('secuestros', conn, if_exists='replace', index=False)


28020

In [43]:
por_anio = pd.read_sql_query("""
SELECT aÑo, SUM(cantidad) AS total_secuestros
FROM secuestros
GROUP BY aÑo
ORDER BY aÑo
""", conn)

print(por_anio)


     aÑo  total_secuestros
0   1996              1038
1   1997              1624
2   1998              2860
3   1999              3205
4   2000              3572
5   2001              2917
6   2002              2882
7   2003              2123
8   2004              1440
9   2005               801
10  2006               690
11  2007               523
12  2008               437
13  2009               213
14  2010               282
15  2011               305
16  2012               305
17  2013               299
18  2014               288
19  2015               213
20  2016               207
21  2017               195
22  2018               176
23  2019               163
24  2020               162
25  2021               160
26  2022               223
27  2023               338
28  2024               305
29  2025               131


In [44]:
por_anio = pd.read_sql_query("""
SELECT anio, SUM(cantidad) AS total_secuestros, departamento
FROM secuestros
GROUP BY aÑO
ORDER BY aÑO
""", conn)

print(por_anio)

DatabaseError: Execution failed on sql '
SELECT anio, SUM(cantidad) AS total_secuestros, departamento
FROM secuestros
GROUP BY aÑO
ORDER BY aÑO
': no such column: anio

In [ ]:
por_dep = pd.read_sql_query("""
SELECT departamento, COUNT(*) as total
FROM secuestros
GROUP BY departamento
ORDER BY total DESC
LIMIT 15
""", conn)
print(por_dep)


          departamento  total
0            ANTIOQUIA   4697
1                CESAR   2123
2      VALLE DEL CAUCA   1965
3          BOGOTA D.C.   1553
4            SANTANDER   1466
5                 META   1372
6   NORTE DE SANTANDER   1355
7         CUNDINAMARCA   1342
8               TOLIMA   1151
9                CAUCA   1088
10             BOLIVAR   1038
11           MAGDALENA   1023
12              NARIÑO    825
13            CASANARE    796
14          LA GUAJIRA    675


In [ ]:
por_anio = pd.read_sql_query("""
SELECT aÑO, SUM(cantidad) AS total_secuestros
FROM secuestros
GROUP BY aÑO
ORDER BY aÑO
""", conn)

print(por_anio)


     aÑo  total_secuestros
0   1996              1038
1   1997              1624
2   1998              2860
3   1999              3205
4   2000              3572
5   2001              2917
6   2002              2882
7   2003              2123
8   2004              1440
9   2005               801
10  2006               690
11  2007               523
12  2008               437
13  2009               213
14  2010               282
15  2011               305
16  2012               305
17  2013               299
18  2014               288
19  2015               213
20  2016               207
21  2017               195
22  2018               176
23  2019               163
24  2020               162
25  2021               160
26  2022               223
27  2023               338
28  2024               305
29  2025               131


In [ ]:
conn.close()
